[![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/jolin-io/fall-in-love-with-julia/main?filepath=11%20meta%20programming%20-%2001%20introduction.ipynb)

<a href="https://www.jolin.io" target="_blank" rel="noreferrer noopener">
<img src="https://www.jolin.io/assets/Jolin/Jolin-Banner-Website-v1.1-darkmode.webp">
</a>

# Fall-in-love-with-Julia: Meta Programming in Julia 101

an introduction session

I am Stephan Sahm, and today we are going to learn all about meta.

1. `Expr` expressions
2. `eval` evaluate expressions
3. `macro` create syntax helpers
4. `@generated` meta functions
5. `Cassette.jl` meta programming without `Expr`
6. `IRTools.jl` generated functions for IR

# `Expr` expressions

Julia is kind of a LISP dialect: Julia's syntax is part of julia itself.
Meta programming is builtin.

In [ ]:
# run some code
result = 1 + 2 + 3 + 4

In [ ]:
# turn it into an expression
expr = :(result)

In [ ]:
# visualize expr
dump(expr)

In [ ]:
# this is what we want
expr = :(1 + 2 + 3 + 4)

In [ ]:
dump(expr)

In [ ]:
expr.head

In [ ]:
expr.args

An alternative to create Expr is using `quote ... end`

In [ ]:
expr = quote
    1 + 2 + 3 + 4
end

But be careful, that it generates an additional block level,
with meta information about where it was constructed

In [ ]:
dump(expr)

You can also built `Expr` programmatically

In [ ]:
expr = Expr(expr.head, expr.args...)

#### it's your time

👉 try `typeof(expr)` on the `expr` we generated so far

In [ ]:
# your space

## Interpolating into Expr

`Expr` is a truly flexible data structure: *You can use any arbitary julia value within `Expr`*

In [ ]:
expr = Expr(:call, +, 1, 2, 3, 4)

In [ ]:
dump(expr)

to combine arbitrary values with `Expr` use interpolation via `$`

In [ ]:
result

In [ ]:
:(2 + result)

In [ ]:
:(2 + $result)

if you want to insert several arguments at once, you can use the splash `...` syntax

In [ ]:
:(+(1,2,3,4))

In [ ]:
arguments = (1, 2, 3, 4)
:(+($(arguments...)))

same can be done within `quote ... end` blocks

# `eval` evaluate expressions

Without the ability to run expressions, they would be pretty useless.

In [ ]:
expr

In [ ]:
eval(expr)

the macro `@eval` version let's you write normal code, but use expr-interpolation within it

In [ ]:
mysymbol = :nice
@eval $mysymbol = "hello world"
nice

Congratulations! Now you are allmighty 💪🦸 

#### it's your time

👉 evaluate a couple of expr

## Eval is scoped to a Module

So far we evaluated into `Main` module.

Note: Also all variables get evaluted as globals into the respective Module.

In [ ]:
Main.nice

In [ ]:
module MyModule
end

In [ ]:
MyModule.eval(:(what_do_we_want = "Climate Justice!"))
MyModule.what_do_we_want

the `@eval` macro is centrally defined and works a little different

In [ ]:
@eval MyModule when_do_we_want_it = "Now!"
MyModule.when_do_we_want_it

## The limitations of eval

Julia combines flexibility with performance. Performance means compiling code, however if you can change code all along, how could it be compiled?

The answer in julia is **"world age"**: Everytime you `eval` a function definition, the world get's older.

In [ ]:
Base.get_world_counter()

In [ ]:
@eval twice(a) = 2a

In [ ]:
Base.get_world_counter()

#### it's your time

👉 try a couple of `eval` to see what increases the world age and what not

In [ ]:
# your space

#### What does world age mean for you?

quote from the paper http://janvitek.org/pubs/oopsla20-j.pdf

> Semantically, newly added methods (i.e. ones defined using eval) only become visible when execution returns to the top level,
and the set of callable methods for an execution is fixed when it leaves the top level.

In my words: Whatever function you create within a function is not visible within the same functioncall.

In [ ]:
function test_eval_global_function()
    @eval general_advise() = "give it a second try and it will work"
    general_advise()
end

In [ ]:
test_eval_global_function()

In [ ]:
function test_eval_global_var()
    @eval another_advise = "if it still doesn't work, change something"
    another_advise
end

In [ ]:
test_eval_global_var()

Impressive: global variables can indeed be constructed on the fly.

But be careful - this is one of the reasons why global variables are bad for performance.

In [ ]:
function test_eval_global_function_cheat()
    @eval cheat() = "if you really need it, this is how to access a just created function"
    Base.invokelatest(cheat)
end

In [ ]:
test_eval_global_function_cheat()

# `macro` create syntax helpers


`macro = Expr + eval`

A macro is a special function which returns an `Expr` that is immediately evaluated.

In [ ]:
macro timeit(expr)
    quote
        before = time()
        result = $expr
        after = time()
        println("timedit: $(after - before) seconds")
        result
    end
end

In [ ]:
@timeit sleep(1)

#### it's your time

👉 time a couple of things

In [ ]:
# your space

#### Macros under the hood

In [ ]:
@macroexpand @timeit sleep(1)

This is called macro hygiene.

New variables defined within the macro are automatically renamed to nonconflicting variables

In [ ]:
macro create_variables1()
    quote
        $(esc(:created_variable1)) = "works" 
    end
end

In [ ]:
@create_variables1
created_variable1

In [ ]:
macro create_variables2()
    @gensym helper
    esc(quote
        $helper = "works too"
        created_variable2 = $helper
    end)
end

In [ ]:
@create_variables2
created_variable2

In [ ]:
@macroexpand @create_variables2

#### it's your time

👉 inspect what `@show expr` and `@gensym helper` are doing

In [ ]:
# your space

## Packages to help you manipulating macros

Macros can be quite nasty to work with. There are a couple of packages which can help you:
- [MacroTools.jl](https://github.com/FluxML/MacroTools.jl) by FluxML - functional tools
- [ExprParsers.jl](https://github.com/jolin-io/ExprParsers.jl) by me - object oriented tools


# `@generated` meta functions

like a macro, but a function

In [ ]:
@generated function twice(x)
    # Within generated functions, normal printing does not work.
    # Think of a generated function as being run at compile-time.
    # Luckily there is a more basic alternative:
    Core.println(x)
    return :(x * x)
end

In [ ]:
twice(2)

In [ ]:
twice(2)

Welcome to the just-in-time version of `eval` 🙂.

#### How to inspect generated functions?

As of now it is not easily possible to grab the generated code from a generated function.

There is even [a stackoverflow question](https://stackoverflow.com/questions/66402105/any-way-to-expand-a-generated-function-in-julia) about it (in julia most often discourse is used instead of stackoverflow).

In [ ]:
@code_lowered twice(2)

#### Real example: Comparing struct types

[StructEquality.jl](https://github.com/jolin-io/StructEquality.jl) is one package of mine which uses @generated functions to solve a simple but common difficulty

In [ ]:
struct MyArray
    value::AbstractArray
end

In [ ]:

["same"] == ["same"]

In [ ]:
MyArray(["same"]) == MyArray(["same"])

this is unintuitive for most

In [ ]:
using StructEquality: @struct_hash_equal, struct_equal

In [ ]:
@struct_hash_equal MyArray

MyArray(["same"]) == MyArray(["same"])

much better

In [ ]:
@macroexpand @struct_hash_equal MyArray

In [ ]:
@code_lowered struct_equal(MyArray(["same"]), MyArray(["same"]))

You can find the implementation [at github](https://github.com/jolin-io/StructEquality.jl/blob/4756b0906ad0fb742f10aad7c5c017226ee2405a/src/StructEquality.jl#L14-L25) (leave a star if you like it)

# `Cassette.jl` meta programming without `Expr`

<p align="center">
<img width="350px" src="https://raw.githubusercontent.com/JuliaLabs/Cassette.jl/master/docs/img/cassette-logo.png"/>
</p>

> Cassette lets you easily extend the Julia language by directly injecting the Julia compiler with new, context-specific behaviors.

In [ ]:
using Cassette
Cassette.@context Ctx;
Cassette.overdub(Ctx(), /, 1, 2)

This was actually an awesome generated function

In [ ]:
@code_lowered 1/2

In [ ]:
@code_lowered Cassette.overdub(Ctx(), /, 1, 2)

As you can see, `overdub` hooks on the original function definition, but rewrites it
such that everything is wrapped into `overdub` and similar helpers. 

How you use `overdub`:

In [ ]:
Cassette.@context SinToCosCtx

# Override the default recursive `overdub` implementation for `sin(x)`.
# Note that there's no tricks here; this is just a normal Julia method
# overload using the normal multiple dispatch semantics.
Cassette.overdub(::SinToCosCtx, ::typeof(sin), x) = -cos(x)

In [ ]:
x = rand(10)
y = Cassette.overdub(SinToCosCtx(), sum, i -> cos(i) + sin(i), x)

#### it's your time

👉 build a `overdub` which ignores `println` statements

In [ ]:
Cassette.@context PrintlnCtx

In [ ]:
# your space

In [ ]:
# test function
function add(a, b)
    println("I'm about to add $a + $b")
    c = a + b
    println("c = $c")
    return c
end

In [ ]:
a = rand(3)
b = rand(3)
add(a, b)

In [ ]:
Cassette.overdub(PrintlnCtx(), add, a, b)

👉 extra challenge: collect all the println statements within the context

For this you need to know that a Cassette Context can receive a `metadata` keyword.

In [ ]:
ctx = PrintlnCtx(metadata = Dict(:key => "value"))
ctx.metadata[:key]

In [ ]:
# your space

I hope you have seen, how simple it is to use `Cassette.jl`. It is a kind of meta programming, without the use of `Expr`.

As `Expr` can easily get messy, I especially like the Cassette approach.

# `IRTools.jl` generated functions for IR

IRTools.jl is part of the Flux ecosystem. It is used for computing Gradients in Zygote.jl

IR stands for Intermediate Representation. We already saw `@code_lowered` returning `CodeInfo`
objects which is such a IR. IRTools.jl offers us an alternative IR, which for some
may be easier to work with.

In [ ]:
function intidentity(a)
    s = 0
    for i in 1:a
        s += 1
    end
    s
end    

In [ ]:
@code_lowered intidentity(3)

In [ ]:
using IRTools

ir = @code_ir intidentity(3)

IRTools allows us to easily access the IR.

In [ ]:
block = IRTools.block(ir, 2)

In [ ]:
IRTools.arguments(block)

In [ ]:
IRTools.branches(block)

In [ ]:
IRTools.predecessors(block)

In [ ]:
IRTools.var(11)

Changing the IR is simple, too.

In [ ]:
ir[IRTools.var(11)] = IRTools.xcall(:+, IRTools.var(8), 2)

Finally, you can make it into a function again

In [ ]:
# first argument to ir is the function type itself, which however is not used here
IRTools.evalir(ir, nothing, 10)

In [ ]:
intidentity2 = IRTools.func(ir)
intidentity2(nothing, 10)

Cassette.jl can do similar things by working on CodeInfo objects. That is called a contextual `pass` in Cassette.jl
See https://julia.mit.edu/Cassette.jl/stable/contextualpass/.

## dynamo

`IRTools.@dynamo` allows you to create something like generated functions, but instead of returning an `Expr`, you return an `IRTools.IR`.

In [ ]:
IRTools.@dynamo function inspect(a...)
    # Within a @dynamo, just like any generated functions, normal printing does not work.
    # Think of it as being run at compile-time.
    # Luckily there is a more basic alternative:
    Core.println(a)
    return IRTools.IR(a...)
end

In [ ]:
# note, the first argument is the function itself
inspect(prod, 1:4)

In [ ]:
inspect(prod, 1:4)

In [ ]:
using MacroTools

IRTools.@dynamo function replace_mul_with_sum(a...)
    ir = IRTools.IR(a...)
    ir = MacroTools.prewalk(ir) do x
        x isa GlobalRef && x.name == :* && return GlobalRef(Base, :+)
        return x
    end
    return ir
end

In [ ]:
replace_mul_with_sum() do 
    1 * 2 * 3 * 4
end

You can even see which IR is returned by your dynamo

In [ ]:
@code_ir replace_mul_with_sum() do 
    1 * 2 * 3 * 4
end
# @code_lowered works too

Yet, it does not work recursively

In [ ]:
replace_mul_with_sum() do 
    prod(1:4)
end

But we can make it work by changing our code similar how Cassette does it, where `overdup` always calls `overdub` again.

In [ ]:
IRTools.@dynamo function replace_mul_with_sum_recursively(a...)
    ir = IRTools.IR(a...)
    ir === nothing && return nothing
    ir = MacroTools.prewalk(ir) do x
        x isa GlobalRef && x.name == :* && return GlobalRef(Base, :+)
        return x
    end
    # new: let's recurse
    for (var, statement) in ir
        MacroTools.isexpr(statement.expr, :call) || continue
        ir[var] = IRTools.xcall(replace_mul_with_sum_recursively, statement.expr.args...)
    end
    return ir
end

In [ ]:
replace_mul_with_sum_recursively() do 
    prod(1:4)
end

In [ ]:
@code_ir replace_mul_with_sum_recursively prod(1:4)  # alternative syntax to see output of dynamo

🥳 Congratulations 🥳 You've build your own Cassette.jl-like meta programming functionality.

## Be careful

It is great to know that such tools exist in case you really need it (e.g. it is used for computing automatic gradients in Zygote).

BUT BE CAUTIOUS. You can easily destroy everything and changing IR is hard to debug in general.

One example: If we would have tried to replace `+` with `*` above (i.e. the other way around), things wouldn't have worked.
For one thing, `sum` has a much more complex implementation, which also uses `-`, which will return something totally weird. For another reason, if you would try to write your own `mysum`, you would probably enter an infinite loop. That is because the standard iterator `1:4` will increase its inner state by using `... + 1`, which would translate to `... * 1`, yielding the same state as before and running forever. 

# Further information
- [julia documentation](https://docs.julialang.org/en/v1/manual/metaprogramming/) about macros and generated functions and co
- [ExprParsers.jl](https://github.com/jolin-io/ExprParsers.jl) for Expr manipulation
- [MacroTools.jl](https://github.com/FluxML/MacroTools.jl) for Expr manipulation
- [Cassette.jl docs](https://julia.mit.edu/Cassette.jl/stable/) for further details on Cassette.jl, especially the [compiler pass injection](https://julia.mit.edu/Cassette.jl/stable/contextualpass/) example 
- [IRTools.jl docs](http://fluxml.ai/IRTools.jl/latest/) for more on intermediate representations. 

# Thank you for your participation

for questions or suggestions please contact me at stephan.sahm@jolin.io


#### Sponsored by [Jolin.io](https://www.jolin.io)

<a href="https://www.jolin.io" target="_blank" rel="noreferrer noopener">
<img src="https://www.jolin.io/assets/Jolin/Jolin-Banner-Website-v1.1-darkmode.webp">
</a>

Jolin.io is an IT-consultancy focussing on Julia

We are there to help you, if you want to
- try out Julia at your company, or
- transition Matlab, Fortran, R, Python, etc. to Julia
- or speed up your existing Julia code